# scRNA-seq Imputation

Data Availability Statement
Tabula Muris data

Smart-seq2 https://doi.org/10.6084/m9.figshare.5715040.v1 ( Consortium, The Tabula Muris, 2017a).

10X Chromium https://doi.org/10.6084/m9.figshare.5715040.v1 ( Consortium, The Tabula Muris, 2017b).

R packages

MAGIC: Rmagic (v0.1.0) https://github.com/KrishnaswamyLab/MAGIC

DrImpute: DrImpute (v1.0) https://github.com/ikwak2/DrImpute

scImpute: scImpute(v0.0.8) https://github.com/Vivianstats/scImpute

SAVER: SAVER(v1.0.0) https://github.com/mohuangx/SAVER

Knn-smooth: knn_smooth.R (Version 2) https://github.com/yanailab/knn-smoothing

Scater: scater(v1.6.3) : https://www.bioconductor.org/packages/release/bioc/html/scater.html

Splatter: splatter(v1.2.2) : https://bioconductor.org/packages/release/bioc/html/splatter.html

Permute: permute(v0.9-4) : https://cran.r-project.org/web/packages/permute/index.html

Python/anaconda packages:

Dca : dca(v0.2.2): https://github.com/theislab/dca

Custom scripts: https://github.com/tallulandrews/F1000Imputation

## Denomising scRNA-seq Data with DCA

### 1. Running the dca package on definite endoderm cells (DECs)

The single cell expression data of DECs are obtained from the GEO website (https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE75748). The dataset includes the RNA expression count data of 1018 single cells from snapshot progenitors.

The paper for analysis of the dataset: Single-cell RNA-seq reveals novel regulators of human embryonic stem cell differentiation to definitive endoderm. https://pubmed.ncbi.nlm.nih.gov/27534536/

In [9]:
# !python -m dca.__main__ data/endoderm/endoderm.csv data/endoderm/

In [13]:
import pandas as pd
import os
cur_dir = os.getcwd()
file_path = cur_dir + '/data/endoderm/endoderm.csv'
endoderm = pd.read_csv(file_path)
endoderm.head()

,Unnamed: 0,H1_Exp1.001,H1_Exp1.002,H1_Exp1.003,H1_Exp1.004,H1_Exp1.006,H1_Exp1.007,H1_Exp1.008,H1_Exp1.009,H1_Exp1.010,...,TB_Batch2.135,TB_Batch2.136,TB_Batch2.137,TB_Batch2.138,TB_Batch2.139,TB_Batch2.140,TB_Batch2.141,TB_Batch2.142,TB_Batch2.143,TB_Batch2.144
0,MKL2,10,162,3,42,0,2,18,0,182,...,364,1,21,1127,2119,5,500,18,472,350
1,CD109,6,2,166,9,7,53,4,64,29,...,15,38,38,11,48,23,362,22,36,25
2,ABTB1,0,28,0,1,0,9,0,0,0,...,0,0,0,0,0,0,0,3,39,0
3,MAST2,0,133,41,0,0,2,0,0,0,...,175,41,32,3,6,206,43,2,1,99
4,KAT5,0,7,52,20,0,6,0,0,103,...,0,577,0,3,2,0,56,2,0,0


In [ ]:
from keras.layers import Dense, Input
from keras.models import Model
from dca.layers import ColwiseMultLayer

input_size = adata.n_vars
output_size = input_size
hid_size = 32
extra_models = {}

def build(adata):
    input_layer = Input(shape=(input_size,), name='count')
    sf_layer = Input(shape=(1,), name='size_factors')

    decoder_output = Dense(hid_size, activation=None, name='center')(input_layer)
    mean = Dense(output_size, name='mean')(decoder_output)
    output = ColwiseMultLayer([mean, sf_layer])

    extra_models['mean_norm'] = Model(inputs=input_layer, outputs=mean)
    extra_models['decoded'] = Model(inputs=input_layer, outputs=decoder_output)
    
    model = Model(inputs=[input_layer, sf_layer], outputs=output)
 
    encoder = Model(inputs=model.input,
                    outputs=model.get_layer('center').output)


    print('dca: Calculating low dimensional representations...')

    adata.obsm['X_dca'] = encoder.predict({'count': adata.X,
                                           'size_factors': adata.obs.size_factors})        
    print('dca: Calculating reconstructions...')

    adata.X = model.predict({'count': adata.X,
                             'size_factors': adata.obs.size_factors})
    
    return adata

adata = build(adata)

## scDMFK

In [50]:
!python scDMFK/run.py --dataname "endoderm/endoderm.csv" --outputdir "endoderm/DMFK-results"

Instructions for updating:
non-resource variables are not supported in the long term
Using TensorFlow backend.
begin training
Successfully preprocessed 1018 genes and 19097 cells.
begin the pretraining
2024-04-11 14:40:22.825348: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 14:40:22.853416: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)
[[ -7.61721039  -4.44107199  -5.94987202 ...   2.3466146   -3.77788305
    1.32349837]
 [-14.36012173  -1.99698162  -4.53384209 ...  -5.54547119   1.26300061
  -18.28737259]
 [-39.85637283 -16.12322235  -5.64176178 ...   8.47264957  18.71442413
  -59.00668335]
 ...
 [  2.1164701   -5.86501551

In [38]:
string = 'data/Young/data.h5'

string[:string.rindex("/")]


'data/Young'

In [8]:
import scDMFK.utils as utils
import numpy as np
import h5py
import scipy as sp
import pandas as pd
import scanpy as sc


def read_clean(data):
    assert isinstance(data, np.ndarray)
    if data.dtype.type is np.bytes_:
        data = utils.decode(data)
    if data.size == 1:
        data = data.flat[0]
    return data

def dict_from_group(group):
    assert isinstance(group, h5py.Group)
    d = utils.dotdict()
    for key in group:
        if isinstance(group[key], h5py.Group):
            value = dict_from_group(group[key])
        else:
            value = read_clean(group[key][...])
        d[key] = value
    return d

def read_data(filename, sparsify=False, skip_exprs=False):
    with h5py.File(filename, "r") as f:
        obs = pd.DataFrame(dict_from_group(f["obs"]), index=utils.decode(f["obs_names"][...]))
        var = pd.DataFrame(dict_from_group(f["var"]), index=utils.decode(f["var_names"][...]))
        uns = dict_from_group(f["uns"])
        if not skip_exprs:
            exprs_handle = f["exprs"]
            if isinstance(exprs_handle, h5py.Group):
                mat = sp.sparse.csr_matrix((exprs_handle["data"][...], exprs_handle["indices"][...],
                                               exprs_handle["indptr"][...]), shape=exprs_handle["shape"][...])
            else:
                mat = exprs_handle[...].astype(np.float32)
                if sparsify:
                    mat = sp.sparse.csr_matrix(mat)
        else:
            mat = sp.sparse.csr_matrix((obs.shape[0], var.shape[0]))
    return mat, obs, var, uns

def prepro(filename):
    data_path = "data/" + filename

    try:
        adata = sc.read(data_path, first_column_names=True)
        adata.obs['Group'] = None
    except:
        mat, obs, var, uns = read_data(data_path, sparsify=False, skip_exprs=False)
        if isinstance(mat, np.ndarray):
            X = np.array(mat)
        else:
            X = np.array(mat.toarray())
        cell_name = np.array(obs["cell_type1"])
        cell_type, cell_label = np.unique(cell_name, return_inverse=True)

        X = np.ceil(X).astype(np.int)
        adata = sc.AnnData(X)
        adata.obs['Group'] = cell_label

    print('Successfully preprocessed {} genes and {} cells.'.format(adata.n_vars, adata.n_obs))
    return adata


def normalize(adata, highly_genes = None, size_factors=True, normalize_input=True, logtrans_input=True):
    sc.pp.filter_genes(adata, min_counts=1)
    sc.pp.filter_cells(adata, min_counts=1)
    if size_factors or normalize_input or logtrans_input:
        adata.raw = adata.copy()
    else:
        adata.raw = adata

    if size_factors:
        sc.pp.normalize_per_cell(adata)
        adata.obs['size_factors'] = adata.obs.n_counts / np.median(adata.obs.n_counts)
    else:
        adata.obs['size_factors'] = 1.0

    if logtrans_input:
        sc.pp.log1p(adata)

    if highly_genes != None:
        sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, n_top_genes = highly_genes, subset=True)

    if normalize_input:
        sc.pp.scale(adata)
    return adata

In [5]:
## Endoderm dataset

# adata = prepro('endoderm/endoderm.csv')
# X = adata.X.astype(np.float32)

# adata = normalize(adata, highly_genes=adata.n_vars)
# sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5, n_top_genes = 10, subset=True)

adata.var.highly_variable.index
pd.DataFrame(adata.X[:, adata.var.highly_variable])

,0,1,2,3,4,5,6,7,8,9,...,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017
0,0.069419,1.468012,-0.502400,0.907161,-0.933539,-0.577798,0.554079,-0.862384,1.470833,-0.649936,...,1.326697,-0.755374,0.076480,2.070367,2.234833,-0.298707,1.671089,0.068661,1.497501,1.442243
1,-0.682182,-0.867690,0.790358,-0.569321,-0.612554,0.236100,-0.575316,0.476853,-0.276732,-0.555143,...,-0.566862,-0.224366,-0.264537,-0.505149,-0.084520,-0.272777,0.824979,-0.378489,-0.226257,-0.304873
2,-0.963323,2.140748,-0.988221,0.405716,-0.933539,1.898989,-0.770293,-0.862384,-1.021800,-0.974608,...,-0.956583,-0.901495,-0.967738,-0.829039,-0.846477,-0.813906,-0.867951,0.541623,1.685979,-0.850209
3,-0.963323,1.994182,1.503586,-0.954341,-0.933539,-0.150747,-0.770293,-0.862384,-1.021800,0.364907,...,1.495311,0.874722,0.760815,-0.124056,0.095633,1.853983,1.020409,-0.333062,-0.536624,1.365221
4,-0.963323,0.180837,1.480572,0.958142,-0.933539,0.288384,-0.770293,-0.862384,1.618809,1.739285,...,-0.956583,1.961002,-0.967738,-0.221934,-0.388543,-0.813906,1.015744,-0.416733,-0.880896,-0.850209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18245,0.763035,0.953469,-0.119517,1.065267,1.176381,1.022391,1.427621,1.002929,0.928195,1.266762,...,0.973501,-0.557952,1.223269,1.731255,0.920591,-0.579516,0.281782,-0.620383,0.399395,0.147709
18246,-0.963323,-0.963068,-0.988221,-0.954341,-0.933539,-0.963207,-0.770293,-0.862384,-1.021800,-0.974608,...,1.960357,1.852234,1.830183,2.251798,1.763214,1.709863,2.262206,-0.875434,0.609419,2.119040
18247,0.721264,1.287970,0.565610,1.260535,-0.451260,0.846659,1.868497,0.597759,0.106420,-0.313491,...,1.345113,0.493171,-0.013193,-0.829039,-0.846477,-0.813906,1.585123,-0.875434,1.402134,-0.850209
18248,0.667025,0.750945,1.359231,1.181274,1.025842,1.324286,0.719553,1.220835,1.090974,0.876491,...,1.145226,1.133138,1.149106,1.105013,1.333495,1.297173,1.085032,0.878802,1.643264,1.311002


In [24]:
# adata = prepro('Young/data.h5')
# (adata.X != 0).any(axis=0)
adata.obs

,Group
0,6
1,6
2,6
3,6
4,6
...,...
5680,0
5681,0
5682,0
5683,0


## Iterative imputation

In [4]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import numpy as np

In [6]:

endoderm = pd.read_csv('data/endoderm/endoderm.csv', index_col=0)
endoderm.head()
endoderm = endoderm.T

In [8]:
data = endoderm.iloc[:50, :50]
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(endoderm, test_size=0.2, random_state=42)

# initialization strategy: mean
%time
imp_mean = IterativeImputer(random_state=0, missing_values = 0)
imp_mean.fit(X_train)
transformed = imp_mean.transform(X_test)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.3 µs


/Users/yufeideng/Documents/GitHub/bis687-project/sc-env/lib/python3.8/site-packages/sklearn/impute/_base.py:558: UserWarning: Skipping features without any observed values: [    7    16    36    42    55    63    68    96    99   132   145   172
   207   210   244   283   290   304   331   334   335   444   450   457
   471   495   539   543   572   580   637   669   672   674   679   716
   736   767   775   929   936   938  1005  1013  1051  1060  1069  1146
  1247  1261  1333  1370  1386  1402  1423  1482  1544  1546  1550  1566
  1597  1605  1637  1642  1645  1686  1714  1729  1782  1786  1789  1826
  1870  1872  1877  1881  1924  1953  1979  1981  2011  2012  2022  2057
  2089  2122  2129  2134  2161  2218  2224  2241  2256  2261  2279  2286
  2294  2331  2381  2384  2385  2395  2452  2485  2487  2492  2523  2528
  2587  2591  2608  2623  2630  2669  2684  2685  2729  2730  2750  2820
  2827  2829  2846  2859  2873  2890  2892  2899  2914  2923  2932  2941
  2984  3037  3041  3077

In [ ]:
transformed